# F-14A CADC Polynomial Verification

This notebook verifies the polynomial computation from the VHDL simulation.

**Polynomial:** E = a₃X³ + a₂X² + a₁X + a₀

**Coefficients (Q1.19 format):**
- a₃ = 0x10000 (0.125)
- a₂ = 0x20000 (0.25)
- a₁ = 0x30000 (0.375)
- a₀ = 0x08000 (0.0625)

In [ ]:
import numpy as np

# Q1.19 format: 1 sign bit, 19 fractional bits
# Range: -1.0 to +0.999998 (approximately)
# Resolution: 2^-19 = 1/524288 ≈ 0.0000019

Q19_SCALE = 2**19  # 524288

def q19_to_float(hex_val):
    """Convert Q1.19 hex value to float"""
    # Handle as unsigned 20-bit, then treat as signed
    if hex_val >= 0x80000:  # Negative (sign bit set)
        return (hex_val - 0x100000) / Q19_SCALE
    return hex_val / Q19_SCALE

def float_to_q19(f):
    """Convert float to Q1.19 hex value"""
    val = int(round(f * Q19_SCALE))
    if val < 0:
        val = val + 0x100000  # Two's complement
    return val & 0xFFFFF

print(f"Q1.19 Scale Factor: {Q19_SCALE}")
print(f"Resolution: {1/Q19_SCALE:.10f}")

In [ ]:
# Data ROM Coefficients from cadc_top.vhd
a3_hex = 0x10000
a2_hex = 0x20000
a1_hex = 0x30000
a0_hex = 0x08000

# Convert to float
a3 = q19_to_float(a3_hex)
a2 = q19_to_float(a2_hex)
a1 = q19_to_float(a1_hex)
a0 = q19_to_float(a0_hex)

print("Polynomial Coefficients:")
print(f"  a3 = 0x{a3_hex:05X} = {a3:.6f}")
print(f"  a2 = 0x{a2_hex:05X} = {a2:.6f}")
print(f"  a1 = 0x{a1_hex:05X} = {a1:.6f}")
print(f"  a0 = 0x{a0_hex:05X} = {a0:.6f}")

In [ ]:
def polynomial(x, a3, a2, a1, a0):
    """Calculate E = a3*X^3 + a2*X^2 + a1*X + a0"""
    return a3 * x**3 + a2 * x**2 + a1 * x + a0

def horner_polynomial(x, a3, a2, a1, a0):
    """Calculate using Horner's method: E = ((a3*X + a2)*X + a1)*X + a0"""
    result = a3
    result = result * x + a2
    result = result * x + a1
    result = result * x + a0
    return result

# Test inputs from VHDL simulation
test_inputs = [
    (0x40000, "0.5"),   # X = 0.5
    (0x20000, "0.25"),  # X = 0.25
    (0x60000, "0.75"),  # X = 0.75
]

print("Expected Results (Floating Point):")
print("="*60)
for x_hex, x_name in test_inputs:
    x = q19_to_float(x_hex)
    e_direct = polynomial(x, a3, a2, a1, a0)
    e_horner = horner_polynomial(x, a3, a2, a1, a0)
    e_hex = float_to_q19(e_horner)
    print(f"X = 0x{x_hex:05X} ({x_name})")
    print(f"  Direct:  E = {e_direct:.6f}")
    print(f"  Horner:  E = {e_horner:.6f}")
    print(f"  Q1.19:   E = 0x{e_hex:05X}")
    print()

In [ ]:
# VHDL Simulation Results from cadc_top_tb
vhdl_results = {
    0x40000: 0x00400,  # X = 0.5  -> E = 0x00400
    0x20000: 0x00400,  # X = 0.25 -> E = 0x00400
    0x60000: 0x00040,  # X = 0.75 -> E = 0x00040
}

print("Comparison: Expected vs VHDL Simulation")
print("="*70)
print(f"{'Input X':<12} {'Expected':<15} {'VHDL Result':<15} {'Match?':<10}")
print("-"*70)

for x_hex, x_name in test_inputs:
    x = q19_to_float(x_hex)
    expected = horner_polynomial(x, a3, a2, a1, a0)
    expected_hex = float_to_q19(expected)
    vhdl_hex = vhdl_results[x_hex]
    vhdl_float = q19_to_float(vhdl_hex)
    
    match = "✓" if expected_hex == vhdl_hex else "✗"
    
    print(f"0x{x_hex:05X} ({x_name:<4}) 0x{expected_hex:05X} ({expected:.4f})  0x{vhdl_hex:05X} ({vhdl_float:.6f})  {match}")

In [ ]:
# Analyze the discrepancy - trace through Horner's method step by step
print("Step-by-Step Horner's Method Trace for X = 0.5")
print("="*60)

x_hex = 0x40000
x = q19_to_float(x_hex)
print(f"Input X = 0x{x_hex:05X} = {x}")
print()

# Step 1: Load a3
acc = a3
print(f"Step 1: ACC = a3 = {acc:.6f} (0x{float_to_q19(acc):05X})")

# Step 2: ACC * X (PMU multiply)
product1 = acc * x
print(f"Step 2: ACC * X = {acc:.6f} * {x} = {product1:.6f} (0x{float_to_q19(product1):05X})")

# Step 3: ACC + a2
acc = product1 + a2
print(f"Step 3: + a2 = {product1:.6f} + {a2:.6f} = {acc:.6f} (0x{float_to_q19(acc):05X})")

# Step 4: ACC * X (PMU multiply)
product2 = acc * x
print(f"Step 4: ACC * X = {acc:.6f} * {x} = {product2:.6f} (0x{float_to_q19(product2):05X})")

# Step 5: ACC + a1
acc = product2 + a1
print(f"Step 5: + a1 = {product2:.6f} + {a1:.6f} = {acc:.6f} (0x{float_to_q19(acc):05X})")

# Step 6: ACC * X (PMU multiply)
product3 = acc * x
print(f"Step 6: ACC * X = {acc:.6f} * {x} = {product3:.6f} (0x{float_to_q19(product3):05X})")

# Step 7: ACC + a0
final = product3 + a0
print(f"Step 7: + a0 = {product3:.6f} + {a0:.6f} = {final:.6f} (0x{float_to_q19(final):05X})")

print()
print(f"Expected Result: 0x{float_to_q19(final):05X} = {final:.6f}")
print(f"VHDL Result:     0x{vhdl_results[x_hex]:05X} = {q19_to_float(vhdl_results[x_hex]):.6f}")

## Analysis

The VHDL simulation results don't match the expected floating-point calculations.

Possible causes:
1. **PMU fractional multiply issue** - The PMU returns upper 20 bits of 40-bit product, which may have different scaling
2. **Data ROM address mapping** - The microprogram may not be reading the correct coefficients
3. **Accumulator path issue** - Data may not be flowing correctly through the steering logic

The consistent result of 0x00400 for different inputs suggests a data path issue.